In [ ]:
import pyproj
import numpy as np
import geoviews as gv
from bokeh.sampledata.airport_routes import airports, routes

gv.extension('bokeh')

## Define data

In [ ]:
def get_circle_path(start, end, samples=200):
    sx, sy = start
    ex, ey = end
    g = pyproj.Geod(ellps='WGS84')
    (az12, az21, dist) = g.inv(sx, sy, ex, ey)
    lonlats = g.npts(sx, sy, ex, ey, samples)
    return np.array([(sx, sy)]+lonlats+[(ex, ey)])

# Compute great-circle paths for all US flight routes from Honolulu
paths = []
honolulu = (-157.9219970703125, 21.318700790405273)
routes = routes[routes.SourceID==3728]
airports = airports[airports.AirportID.isin(list(routes.DestinationID)+[3728])]
for i, route in routes.iterrows():
    airport = airports[airports.AirportID==route.DestinationID].iloc[0]
    paths.append(get_circle_path(honolulu, (airport.Longitude, airport.Latitude)))
tiles = gv.tile_sources.Wikipedia

# Define Graph from Nodes and EdgePaths
path = gv.EdgePaths(paths)
points = gv.Nodes(airports, ['Longitude', 'Latitude', 'AirportID'], ['Name', 'City'])
graph = gv.Graph((routes, points, path), ['SourceID', 'DestinationID'])

## Plot

In [ ]:
tiles * graph.opts(width=600, height=400, tools=['hover', 'tap'], node_color='black')